In [2]:
import pandas as pd
import numpy as np

In [17]:
# display 확장
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

# train, valid, test file 경로 지정
file = './physionet2012_train.csv'
file2 = './physionet2012_valid.csv'
file3 = './physionet2012_test.csv'

# train data 로딩
df = pd.read_csv(file, index_col=None)
del df['Unnamed: 0']
df.head()

# valid data 로딩
valid_df = pd.read_csv(file2, index_col=None)
del valid_df['Unnamed: 0']
valid_df.head()

# tess data 로딩
test_df = pd.read_csv(file3, index_col=None)
del test_df['Unnamed: 0']
test_df.head()

,pid,rec_time,age,Gender,ICUType,ALP,ALT,AST,Albumin,BUN,Bilirubin,Cholesterol,Creatinine,FiO2,GCS,Glucose,HCO3,HCT,HR,K,Lactate,Mg,Na,PaCO2,PaO2,Platelets,RR,SPO2,BT,Tropl,TroponinI,TropT,TroponinT,Urine,WBC,pH,MechVent,survival,SBP,DBP,MBP,male,female,surv_los
0,135089,0:00:00,90.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,97.0,37.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,116.0,58.0,77.0,1,0,-7
1,135089,0:44:00,90.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.0,NaN,2.9,NaN,NaN,NaN,NaN,NaN,19.0,97.0,37.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,116.0,58.0,77.0,1,0,-7
2,135089,2:09:00,90.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,130.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,97.0,38.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,116.0,58.0,77.0,1,0,-7
3,135089,2:10:00,90.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,130.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,97.0,38.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,116.0,58.0,77.0,1,0,-7
4,135089,2:15:00,90.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,97.0,38.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,116.0,58.0,77.0,1,0,-7


In [18]:
def calculate_mews(respiratory_rate, heart_rate, systolic_bp, temperature):
    mews = 0
    
    # Respiratory Rate
    if respiratory_rate >= 25:
        mews += 3
    elif respiratory_rate >= 21:
        mews += 2
    elif respiratory_rate >= 11:
        mews += 1
    
    # Heart Rate
    if heart_rate >= 130:
        mews += 3
    elif heart_rate >= 110:
        mews += 2
    elif heart_rate >= 91:
        mews += 1
    
    # Systolic Blood Pressure
    if systolic_bp < 90:
        mews += 2
    elif systolic_bp < 100:
        mews += 1
    
    # Temperature
    if temperature < 35 or temperature >= 39:
        mews += 2
    
    
    return mews

In [21]:
df

,pid,rec_time,age,Gender,ICUType,ALP,ALT,AST,Albumin,BUN,Bilirubin,Cholesterol,Creatinine,FiO2,GCS,Glucose,HCO3,HCT,HR,K,Lactate,Mg,Na,PaCO2,PaO2,Platelets,RR,SPO2,BT,Tropl,TroponinI,TropT,TroponinT,Urine,WBC,pH,MechVent,survival,SBP,DBP,MBP,male,female,surv_los
0,142219,0:00:00,52.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,97.0,37.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,116.0,58.0,77.0,1,0,-8
1,142219,0:44:00,52.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8,NaN,NaN,NaN,NaN,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,97.0,37.1,NaN,NaN,NaN,NaN,500.0,NaN,NaN,NaN,-1,116.0,58.0,77.0,1,0,-8
2,142219,1:14:00,52.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,97.0,37.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,0.0,0.0,224.0,1,0,-8
3,142219,1:44:00,52.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,97.0,37.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,0.0,0.0,135.0,1,0,-8
4,142219,1:54:00,52.0,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,97.0,37.1,NaN,NaN,NaN,NaN,450.0,NaN,NaN,NaN,-1,217.0,108.0,133.0,1,0,-8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242608,138581,"1 day, 20:36:00",53.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4,8.0,NaN,NaN,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,100.0,36.9,NaN,NaN,NaN,NaN,80.0,NaN,NaN,1.0,17,114.0,65.0,84.0,1,0,-1
242609,138581,"1 day, 21:36:00",53.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,100.0,36.9,NaN,NaN,NaN,NaN,40.0,NaN,NaN,NaN,17,111.0,61.0,80.0,1,0,-1
242610,138581,"1 day, 22:36:00",53.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,100.0,36.9,NaN,NaN,NaN,NaN,45.0,NaN,NaN,NaN,17,109.0,62.0,80.0,1,0,-1
242611,138581,"1 day, 23:36:00",53.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,100.0,36.9,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,17,111.0,63.0,80.0,1,0,-1


In [11]:
first_index = test_df.drop_duplicates('pid').index

In [ ]:
first_index

In [10]:
cols = ['RR', 'RR', 'SBP', 'BT']

In [ ]:
test_df.columns

In [6]:
first_index = df.drop_duplicates('pid').index
first_indext = test_df.drop_duplicates('pid').index
first_indexv= valid_df.drop_duplicates('pid').index

colss=df.columns
cols=[]
for i in range (2,colss.size):
    cols.append(colss[i])
# 중앙값 구하기
median_dict={}
stat_dict = {}
for col in cols:
    median_dict[col] = df[col].median()
    stat_dict[col] = {'mean': df[col].mean(), 'std': df[col].std()}

In [7]:
median_dict

{'age': 67.0,
 'Gender': 1.0,
 'ICUType': 3.0,
 'ALP': 82.0,
 'ALT': 43.0,
 'AST': 63.0,
 'Albumin': 2.9,
 'BUN': 20.0,
 'Bilirubin': 0.9,
 'Cholesterol': 154.0,
 'Creatinine': 1.0,
 'FiO2': 0.5,
 'GCS': 13.0,
 'Glucose': 127.0,
 'HCO3': 23.0,
 'HCT': 30.3,
 'HR': 86.0,
 'K': 4.1,
 'Lactate': 2.1,
 'Mg': 2.0,
 'Na': 139.0,
 'PaCO2': 39.0,
 'PaO2': 122.0,
 'Platelets': 173.0,
 'RR': 19.0,
 'SPO2': 97.0,
 'BT': 37.1,
 'Tropl': nan,
 'TroponinI': 2.9,
 'TropT': nan,
 'TroponinT': 0.22,
 'Urine': 70.0,
 'WBC': 11.5,
 'pH': 7.38,
 'MechVent': 1.0,
 'survival': -1.0,
 'SBP': 116.0,
 'DBP': 58.0,
 'MBP': 77.0,
 'male': 1.0,
 'female': 0.0,
 'surv_los': -7.0}

In [13]:
test_df = test_df3

In [8]:
test_df3 = test_df.copy()
# test 데이터
for col in cols:
    for i in first_indext:
        if np.isnan(test_df3.loc[i,col])==True:
            test_df3.loc[i, col] = median_dict[col]
    test_df3[col] = test_df3[col].fillna(method='ffill')

In [ ]:
test_df.loc[first_index, col]

In [9]:
from tqdm import tqdm

In [14]:
mews_scores=[]
for i in tqdm(test_df.index):
    data = test_df.loc[i, cols].values
    mews_score = calculate_mews(data[0], data[1], data[2], data[3])
    mews_scores.append(mews_score)
test_df["MEWS"] = mews_scores

100%|██████████| 29554/29554 [00:11<00:00, 2486.42it/s]


In [15]:
test_df.MEWS

0        1
1        1
2        3
3        2
4        2
        ..
29549    2
29550    2
29551    1
29552    1
29553    1
Name: MEWS, Length: 29554, dtype: int64

In [16]:
test_df.to_csv("mews.csv")

In [5]:
first_index = test_df.drop_duplicates('pid').index
cols = ['RR', 'RR', 'SBP', 'BT']
median_dict = {}
for col in cols:
    median_dict[col] = test_df[col].median()
for col in cols:
    test_df.loc[first_index, col] = median_dict[col]
    test_df[col]=test_df[col].fillna(method='ffill')

from tqdm import tqdm
mews_scores=[]
for i in tqdm(test_df.index):
    data = test_df.loc[i, cols].values
    mews_score = calculate_mews(data[0], data[1], data[2], data[3])
    mews_scores.append(mews_score)
test_df["MEWS"] = mews_scores
print(test_df.MEWS)

100%|██████████████████████████████████████████████████████████████████████████| 29554/29554 [00:11<00:00, 2524.91it/s]

0        1
1        1
2        3
3        2
4        2
        ..
29549    2
29550    2
29551    1
29552    1
29553    1
Name: MEWS, Length: 29554, dtype: int64


In [6]:
test_df.to_csv("mews.csv")